In [1]:
import pandas as pd
import time
import numpy as np

In [2]:
sales_data = pd.read_csv('/Users/guybasson/Desktop/sdatta-nlp/palmers_fashion/f_sales_v_fashion.csv')
sales_data['date'] = pd.to_datetime(sales_data['date'])
sales_data['item'] = sales_data['sku'].astype(str)
sales_data['store'] = sales_data['store'].astype(str)
sales_data = sales_data.rename(columns={'total_sales':'sales'})
sales_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4990977 entries, 0 to 4990976
Data columns (total 7 columns):
 #   Column                   Dtype         
---  ------                   -----         
 0   sku                      int64         
 1   store                    object        
 2   date                     datetime64[ns]
 3   average_price            float64       
 4   average_dicounted_price  float64       
 5   sales                    float64       
 6   item                     object        
dtypes: datetime64[ns](1), float64(3), int64(1), object(2)
memory usage: 266.5+ MB


In [3]:
sales_data = sales_data[sales_data['sku'] == 100630105000002]

In [4]:
warehouse_data = pd.read_csv('/Users/guybasson/Desktop/sdatta-nlp/palmers_fashion/warehouse_stock_fashion.csv')
warehouse_data['valid_to_date'] = warehouse_data['valid_to_date'].replace('2099-12-31', sales_data['date'].max().strftime('%Y-%m-%d'))

In [5]:
warehouse_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1356889 entries, 0 to 1356888
Data columns (total 5 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   store            1356889 non-null  object 
 1   sku              1356889 non-null  int64  
 2   valid_from_date  1356889 non-null  object 
 3   valid_to_date    1356889 non-null  object 
 4   stock            1356889 non-null  float64
dtypes: float64(1), int64(1), object(3)
memory usage: 51.8+ MB


In [6]:
warehouse_data = warehouse_data[warehouse_data['sku'] == 100630105000002]

In [7]:
warehouse_data['valid_to_date'].max()

'2022-03-25'

In [8]:
mbew_fashion = pd.read_csv('mbew_fashion.csv')
mbew_fashion['valid_to_date'] = mbew_fashion['valid_to_date'].replace('2099-12-31', sales_data['date'].max().strftime('%Y-%m-%d'))
mbew_fashion['valid_to_date'] = pd.to_datetime(mbew_fashion['valid_to_date'])
mbew_fashion['valid_from_date'] = pd.to_datetime(mbew_fashion['valid_from_date'])
mbew_fashion['item'] = mbew_fashion['sku'].astype(str).str[:12]
mbew_fashion

,store,sku,valid_from_date,valid_to_date,stock,item
0,123,100549055000004,2017-04-21,2022-03-25,0.0,100549055000
1,89,100532175000001,2017-04-21,2022-03-25,0.0,100532175000
2,43,100553018000001,2017-04-21,2022-03-25,0.0,100553018000
3,104,100548169000007,2017-04-21,2022-03-25,0.0,100548169000
4,85,100549049000003,2017-04-21,2022-03-25,0.0,100549049000
...,...,...,...,...,...,...
17812117,18,100552850000002,2020-05-07,2022-03-25,0.0,100552850000
17812118,99,100653096000013,2020-05-07,2020-08-14,1.0,100653096000
17812119,46,100557049000004,2018-11-17,2022-03-25,0.0,100557049000
17812120,8,100511203000004,2017-04-21,2022-03-25,0.0,100511203000


In [9]:
mbew_fashion.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17812122 entries, 0 to 17812121
Data columns (total 6 columns):
 #   Column           Dtype         
---  ------           -----         
 0   store            int64         
 1   sku              int64         
 2   valid_from_date  datetime64[ns]
 3   valid_to_date    datetime64[ns]
 4   stock            float64       
 5   item             object        
dtypes: datetime64[ns](2), float64(1), int64(2), object(1)
memory usage: 815.4+ MB


In [10]:
mbew_fashion = mbew_fashion[mbew_fashion['sku'] == 100630105000002]

In [11]:
mbew_fashion_no_zero = mbew_fashion[mbew_fashion['stock'] > 0]
mbew_fashion_no_zero['valid_from_date'] = pd.to_datetime(mbew_fashion_no_zero['valid_from_date'])
dict_of_first_month_date_for_sku = {}
i= 0
for sku in mbew_fashion_no_zero['sku'].unique():
    i+=1
    print(str(i) + ' out of ' + str(len(mbew_fashion_no_zero['sku'].unique())))
    data_ = mbew_fashion_no_zero[mbew_fashion_no_zero['sku'] == sku]
    first_date = data_['valid_from_date'].min()
    # put the month in the dict from the first date
    dict_of_first_month_date_for_sku[sku] =  first_date.strftime('%m/%y')

1 out of 1


/var/folders/sv/p02my_xx7t193mwjyb2ytt040000gn/T/ipykernel_52943/154987957.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mbew_fashion_no_zero['valid_from_date'] = pd.to_datetime(mbew_fashion_no_zero['valid_from_date'])


In [12]:
# take MM/YY from first date
first_date.strftime('%m/%y')


'09/20'

In [13]:
dict_of_first_day_for_sku

NameError: name 'dict_of_first_day_for_sku' is not defined

In [14]:
print(sales_data)

                     sku store       date  average_price  \
2977904  100630105000002    10 2020-09-15          24.99   
2977905  100630105000002    10 2020-09-16          24.99   
2977906  100630105000002    10 2020-09-19          24.99   
2977907  100630105000002    10 2020-09-24          24.99   
2977908  100630105000002    10 2020-10-01          24.99   
...                  ...   ...        ...            ...   
2979597  100630105000002    99 2021-03-03          24.99   
2979598  100630105000002    99 2021-03-12          24.99   
2979599  100630105000002    99 2021-03-13          24.99   
2979600  100630105000002    99 2021-05-03          24.99   
2979601  100630105000002    99 2021-06-05          24.99   

         average_dicounted_price  sales             item  
2977904                    24.99    1.0  100630105000002  
2977905                    24.99    1.0  100630105000002  
2977906                    24.99    1.0  100630105000002  
2977907                    14.99    1.0  10

In [15]:
strong_sku = sales_data.groupby(['sku'])['sales'].sum().sort_values(ascending=False).reset_index()

In [16]:
import matplotlib.pyplot as plt
for sku in [100630490000001, 100630490000002, 100630490000003, 100630490000004, 100630490000005, 100630490000006, 100630490000007, 100630490000008, 100630490000009, 100630490000010, 100630490000011, 100630490000012, 100630490000013, 100630490000014, 100630490000015, 100630490000016, 100630490000017, 100630490000018, 100630490000019, 100630490000020, 100630490000021,]:
    sku_data = sales_data[sales_data['sku'] == sku].groupby('date')['sales'].sum()
    sku_data = sku_data.reindex(pd.date_range(start=sales_data['date'].min(),
                                                # today
                                                end=sales_data['date'].max(),

                                                freq='D')).fillna(0)
    sku_warehouse = warehouse_data[warehouse_data['sku'].astype(str) == str(sku)].sort_values(by='valid_from_date')
    sku_warehouse_df_final = pd.DataFrame()
    for row in sku_warehouse.iterrows():
        sku_warehouse_df_final = pd.concat([sku_warehouse_df_final, pd.DataFrame({'date':pd.date_range(start=row[1]['valid_from_date'], end=row[1]['valid_to_date'], freq='D'), 'warehouse stock':row[1]['stock']})])
    sku_warehouse_df_final = sku_warehouse_df_final.set_index('date')
    relevant_sku_stores_stock = mbew_fashion[mbew_fashion['sku'] == sku]
    all_stores_sku_stock_data = {}
    for store in relevant_sku_stores_stock['store'].unique():
        store_sku_data = relevant_sku_stores_stock[relevant_sku_stores_stock['store'] == store]
        one_store_stock_sku_all = pd.DataFrame()
        for row in store_sku_data.iterrows():
            one_store_stock_sku = pd.DataFrame({'date':pd.date_range(start=row[1]['valid_from_date'],
                                                                     end=row[1]['valid_to_date'],
                                                                     freq='D'),
                                                'store stock':row[1]['stock']})
            one_store_stock_sku_all = pd.concat([one_store_stock_sku_all, one_store_stock_sku])
        one_store_stock_sku_all = one_store_stock_sku_all.set_index('date')
        all_stores_sku_stock_data[store] = one_store_stock_sku_all
        # merge all stores with outer join
    all_stores_final_stock_data = pd.DataFrame()
    for store in all_stores_sku_stock_data.keys():
        relevant_store_data_1 = all_stores_sku_stock_data[store]
        relevant_store_data_1 = relevant_store_data_1.rename(columns={'store stock':'store {}'.format(store) + ' stock'})
        all_stores_final_stock_data = pd.concat([all_stores_final_stock_data, relevant_store_data_1], axis=1, join='outer')
    all_store_sum_stock = all_stores_final_stock_data.sum(axis=1).to_frame().rename(columns={0:'all stores stock'})
    # merge by index
    sku_data = pd.merge(sku_data, sku_warehouse_df_final, left_index=True, right_index=True, how='left').fillna(0)
    sku_data = pd.merge(sku_data, all_store_sum_stock, left_index=True, right_index=True, how='left').fillna(0)
    sku_data_w = sku_data.resample('W').agg({'sales':'sum', 'warehouse stock':'mean', 'all stores stock':'mean'})
    sku_data_w.plot(figsize=(20,4))
    plt.title('sku: ' + str(sku))

    plt.show()

KeyError: "None of ['date'] are in the columns"

In [17]:
sku_warehouse = warehouse_data[warehouse_data['sku'].astype(str) == str(sku)].sort_values(by='valid_from_date')

In [18]:
sales_data['sku_store'] = sales_data['sku'].astype(str) + ',' + sales_data['store'].astype(str)
mbew_fashion['sku_store'] = mbew_fashion['sku'].astype(str) + ',' + mbew_fashion['store'].astype(str)

/var/folders/sv/p02my_xx7t193mwjyb2ytt040000gn/T/ipykernel_52943/3695414943.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mbew_fashion['sku_store'] = mbew_fashion['sku'].astype(str) + ',' + mbew_fashion['store'].astype(str)


In [19]:
import numpy as np
import time

start_time = time.time()
mbew_fashion = mbew_fashion.sort_values('valid_from_date')

for store in sales_data['store'].unique():
    print("store:", store)
    len_of_store = sales_data[sales_data['store'] == store]["sku_store"].nunique()

    unique_sku_stores =  sales_data[sales_data['store'] == store]["sku_store"].unique()

    mbew_fashion['valid_from_date'] = pd.to_datetime(mbew_fashion['valid_from_date'])
    mbew_fashion['valid_to_date'] = pd.to_datetime(mbew_fashion['valid_to_date'])
    filtered_mbew_fashion = mbew_fashion[mbew_fashion['sku_store'].isin(unique_sku_stores)]

    # Function to generate date ranges
    def generate_date_ranges(row):
        return pd.date_range(row['valid_from_date'], row['valid_to_date'])

    # Apply function to create date ranges
    df_all_2 = filtered_mbew_fashion.apply(generate_date_ranges, axis=1)

    # Create DataFrame with SKU-store and dates
    df_all_2 = pd.DataFrame({
        'sku_store': np.repeat(filtered_mbew_fashion['sku_store'].values, df_all_2.str.len()),
        'date': np.concatenate(df_all_2.values)  # Convert DatetimeIndex to array for concatenation
    })
    # merge left by ['sku_store', 'date'] and right by ['sku_store', 'valid_to_date']
    df_all_2 = pd.merge(df_all_2, filtered_mbew_fashion[['sku_store','valid_from_date', 'stock']], left_on=['sku_store', 'date'], right_on=['sku_store', 'valid_from_date'], how='left')
    # ffil stock
    df_all_2['stock'] = df_all_2['stock'].ffill()
    df_all_2 = df_all_2.drop(columns=['valid_from_date'])
    df_all_2 = pd.merge(sales_data, df_all_2, on=["sku_store","date"], how="right")
    df_all_2['sku'] = df_all_2['sku_store'].str.split(',').str[0]
    df_all_2['store'] = df_all_2['sku_store'].str.split(',').str[1]
    df_all_2['item'] = df_all_2['sku'].astype(str).str[:12]
    df_all_2['sales'] = df_all_2['sales'].fillna(0)
    df_all_2.to_parquet("/Users/guybasson/Desktop/sdatta-nlp/palmers_fashion/reinforcment_codes/datasets_3/df_all_store_{}.parquet".format(store))

print("--- %s seconds ---" % (time.time() - start_time))

store: 10
store: 100
store: 104
store: 105
store: 106
store: 109
store: 11
store: 114
store: 117
store: 119
store: 121
store: 122
store: 123
store: 130
store: 133
store: 135
store: 136
store: 141
store: 143
store: 144
store: 147
store: 149
store: 15
store: 150
store: 152
store: 156
store: 159
store: 160
store: 162
store: 163
store: 164
store: 166
store: 167
store: 168
store: 170
store: 171
store: 172
store: 173
store: 174
store: 175
store: 179
store: 18
store: 180
store: 181
store: 182
store: 183
store: 184
store: 185
store: 186
store: 188
store: 189
store: 201
store: 202
store: 203
store: 21
store: 213
store: 214
store: 215
store: 216
store: 217
store: 218
store: 219
store: 22
store: 220
store: 221
store: 225
store: 26
store: 27
store: 28
store: 29
store: 3
store: 3005
store: 3202
store: 3205
store: 3208
store: 3245
store: 35
store: 36
store: 37
store: 4
store: 4104
store: 4123
store: 4129
store: 4133
store: 4134
store: 42
store: 43
store: 44
store: 45
store: 46
store: 47
store: 4803


# generate warehouse data

In [ ]:
sku_unique_in_sales = sales_data['sku'].unique()

In [ ]:
print(warehouse_data[warehouse_data['sku'].isin([100642246000001])])

In [ ]:

start_time = time.time()
warehouse_data['valid_from_date'] = pd.to_datetime(warehouse_data['valid_from_date'])
warehouse_data['valid_to_date'] = pd.to_datetime(warehouse_data['valid_to_date'])
warehouse_data = warehouse_data.sort_values('valid_from_date')
warehouse_data['sku_store'] = warehouse_data['sku'].astype(str) + ',' + warehouse_data['store'].astype(str)
warehouse_data_sample = warehouse_data[warehouse_data['sku'].isin(sku_unique_in_sales)]
for store in ['VZ01']:
    print("store:", store)

    warehouse_data_sample['valid_from_date'] = pd.to_datetime(warehouse_data_sample['valid_from_date'])
    warehouse_data_sample['valid_to_date'] = pd.to_datetime(warehouse_data_sample['valid_to_date'])
    filtered_warehouse_data = warehouse_data_sample

    # Function to generate date ranges
    def generate_date_ranges(row):
        return pd.date_range(row['valid_from_date'], row['valid_to_date'])

    # Apply function to create date ranges
    df_all_2 = filtered_warehouse_data.apply(generate_date_ranges, axis=1)

    # Create DataFrame with SKU-store and dates
    df_all_2 = pd.DataFrame({
        'sku': np.repeat(filtered_warehouse_data['sku'].values, df_all_2.str.len()),
        'date': np.concatenate(df_all_2.values)  # Convert DatetimeIndex to array for concatenation
    })
    df_all_2 = pd.merge(df_all_2, filtered_warehouse_data[['sku','valid_from_date', 'stock']], left_on=['sku', 'date'], right_on=['sku', 'valid_from_date'], how='left')
    # ffil stock
    df_all_2['stock'] = df_all_2['stock'].ffill()
  #  df_all_2 = df_all_2.rename(columns={'stock':'warehouse stock'})
    df_all_2 = df_all_2.drop(columns=['valid_from_date'])

    df_all_2.to_parquet("df_all_store_{}.parquet".format(store))

print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
df_all_2.to_parquet("df_all_store_{}.parquet".format(store))

In [ ]:
df_all_2['sku_store'].unique()

In [ ]:
df_all_2

In [ ]:
warehouse_data_sample[warehouse_data_sample['sku'] == '100657013000006']

In [ ]:
unique_sku_stores

In [ ]:
df_all_2

In [ ]:
df_all_2

In [ ]:
mbew_fashion[mbew_fashion['sku_store'] == '100090812000001,100']

In [ ]:
df_all_2[df_all_2['sku_store'] == '100090812000001,100']

In [ ]:
warehouse_data.info()

In [ ]:
sku_warehouse_df_final

In [ ]:
print(100630490000005)

In [ ]:
sku_data